In [16]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from collections import Counter
import cv2
from tqdm import tqdm

# 이미지 파일을 읽어오는 함수
def imread_kor(filePath, mode=cv2.IMREAD_UNCHANGED):
    stream = open(filePath.encode("utf-8"), "rb")
    bytes = bytearray(stream.read())
    numpyArray = np.asarray(bytes, dtype=np.uint8)
    return cv2.imdecode(numpyArray, mode)

# 데이터셋 무개림성 체크 및 CSV 생성 함수 (train/val/test 구조)
def check_dataset_integrity_and_generate_csv(root_dir, output_csv_path, dataset_names=None):
    data_records = []

    # 제공된 dataset_names가 없을 경우 모든 폴더를 대상으로 처리
    if dataset_names is None or len(dataset_names) == 0:
        datasets = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    else:
        datasets = [d for d in dataset_names if os.path.isdir(os.path.join(root_dir, d))]

    for dataset in datasets:
        dataset_path = os.path.join(root_dir, dataset)
        
        # train, val, test 폴더 확인
        splits = ['train', 'val', 'test']
        available_splits = []
        available_splits_nums = []
        for split in splits:
            split_path = os.path.join(dataset_path, split)
            if os.path.exists(split_path):
                original_dir = os.path.join(split_path, 'Originals')
                mask_dir = os.path.join(split_path, 'Masks')
                if os.path.exists(original_dir) and os.path.exists(mask_dir):
                    available_splits.append(split)
                    available_splits_nums.append(len(os.listdir(original_dir)))
        
        if not available_splits:
            print(f"Dataset '{dataset}' has no valid train/val/test splits with Originals and Masks directories. Skipping...\n")
            data_records.append({
                'Dataset Name': dataset,
                'Remark': 'No valid train/val/test splits found with Originals and Masks directories'
            })
            continue

        # 모든 split의 파일들을 수집
        all_original_files = []
        all_mask_files = []
        all_original_paths = []
        all_mask_paths = []
        
        for split in available_splits:
            original_dir = os.path.join(dataset_path, split, 'Originals')
            mask_dir = os.path.join(dataset_path, split, 'Masks')
            
            original_files = sorted([f for f in os.listdir(original_dir) if os.path.isfile(os.path.join(original_dir, f))])
            mask_files = sorted([f for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))])
            
            # 파일 이름(확장자 제외) 비교를 위한 리스트 생성
            original_names = sorted([os.path.splitext(f)[0] for f in original_files])
            mask_names = sorted([os.path.splitext(f)[0] for f in mask_files])
            
            # 각 split 내에서 파일 이름이 일치하는지 확인
            if original_names != mask_names:
                print(f"Dataset '{dataset}' split '{split}' has filename mismatch between Originals and Masks. Skipping this split...")
                continue
            
            # 파일 개수가 일치하는지 확인
            if len(original_files) != len(mask_files):
                print(f"Dataset '{dataset}' split '{split}' has file count mismatch between Originals and Masks. Skipping this split...")
                continue
            
            # 전체 리스트에 추가
            for orig_file, mask_file in zip(original_files, mask_files):
                all_original_files.append(orig_file)
                all_mask_files.append(mask_file)
                all_original_paths.append(os.path.join(original_dir, orig_file))
                all_mask_paths.append(os.path.join(mask_dir, mask_file))

        if not all_original_files:
            print(f"Dataset '{dataset}' has no valid file pairs across all splits. Skipping...\n")
            data_records.append({
                'Dataset Name': dataset,
                'Remark': 'No valid file pairs found across all splits'
            })
            continue

        # 통계 수집을 위한 카운터 초기화
        original_channel_counter = Counter()
        mask_channel_counter = Counter()
        original_shape_counter = Counter()
        mask_shape_counter = Counter()
        extension_counter_originals = Counter()
        extension_counter_masks = Counter()
        unique_class_counter = Counter()
        all_classes = set()

        original_dtype = None
        original_max_value = None
        original_min_value = None
        total_samples = len(all_original_files)

        min_width, max_width = None, None
        min_height, max_height = None, None
        in_channels = None

        # 모든 파일에 대해 데이터 수집
        for original_path, mask_path, original_file, mask_file in tqdm(
            zip(all_original_paths, all_mask_paths, all_original_files, all_mask_files), 
            desc=f"Processing {dataset}", 
            total=len(all_original_files), 
            leave=False
        ):
            # 파일 확장자 추적
            original_ext = os.path.splitext(original_file)[1].lower()
            mask_ext = os.path.splitext(mask_file)[1].lower()
            extension_counter_originals[original_ext] += 1
            extension_counter_masks[mask_ext] += 1

            # 이미지 또는 npy 배열 로드
            if original_file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif')):
                original_image = imread_kor(original_path)
                original_array = np.array(original_image)
            elif original_file.endswith('.npy'):
                original_array = np.load(original_path)
            else:
                continue  # 지원되지 않는 파일 형식

            if mask_file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif')):
                mask_image = imread_kor(mask_path)
                mask_array = np.array(mask_image)
            elif mask_file.endswith('.npy'):
                mask_array = np.load(mask_path)
            else:
                continue  # 지원되지 않는 파일 형식

            # Shape 추적
            original_shape = original_array.shape if len(original_array.shape) == 3 else (*original_array.shape, 1)
            mask_shape = mask_array.shape if len(mask_array.shape) == 3 else (*mask_array.shape, 1)
            original_shape_counter[original_shape] += 1
            mask_shape_counter[mask_shape] += 1

            # 채널 수 추적
            original_channels = original_shape[2]
            mask_channels = mask_shape[2]
            original_channel_counter[original_channels] += 1
            mask_channel_counter[mask_channels] += 1

            # 첫 번째 이미지에서 Inchannels 설정
            if in_channels is None:
                in_channels = original_channels

            # Originals의 데이터 타입, 최대값, 최소값 추적
            if original_dtype is None:
                original_dtype = original_array.dtype
            if original_max_value is None:
                original_max_value = np.max(original_array)
            else:
                original_max_value = max(original_max_value, np.max(original_array))
            if original_min_value is None:
                original_min_value = np.min(original_array)
            else:
                original_min_value = min(original_min_value, np.min(original_array))

            # 마스크의 고유 클래스 추적
            unique_values = tuple(np.unique(mask_array))
            unique_class_counter[unique_values] += 1
            all_classes.update(unique_values)

            # Width 및 Height 최소/최대 값 추적
            height, width = original_array.shape[:2]
            if min_width is None or width < min_width:
                min_width = width
            if max_width is None or width > max_width:
                max_width = width
            if min_height is None or height < min_height:
                min_height = height
            if max_height is None or height > max_height:
                max_height = height

        # CSV에 기록할 행 생성
        original_shape_report = f"Min Height: {min_height}, Max Height: {max_height}, Min Width: {min_width}, Max Width: {max_width}" if len(original_shape_counter) >= 2 else str(dict(original_shape_counter))
        mask_shape_report = f"Min Height: {min_height}, Max Height: {max_height}, Min Width: {min_width}, Max Width: {max_width}" if len(mask_shape_counter) >= 2 else str(dict(mask_shape_counter))
        total_classes = len(all_classes)

        # Unique values 출력
        print(f"Dataset: {dataset} - Unique mask classes: {sorted(list(all_classes))}")
        print(f"Available splits: {available_splits}")

        # CSV 행 생성
        row = {
            'Dataset Name': dataset,
            'Available Splits': str(available_splits),  # 사용 가능한 split 정보 추가
            'Samples per Split': str(available_splits_nums),
            'Image File Extension': str(dict(extension_counter_originals)),
            'Image Shape (H,W)': original_shape_report,
            'Pixel Range': f"{original_min_value} - {original_max_value}",
            'Image Data Type': str(original_dtype),
            'Mask File Extension': str(dict(extension_counter_masks)),
            'Mask Shape (H,W)': mask_shape_report,
            'Number of Samples': total_samples,
            'Inchannels': in_channels,
            'Number of Classes': total_classes,
            'Unique Classes': str(sorted(list(all_classes))),
            'Remark': '',  # 이슈가 없을 경우 빈값
            'Transform':'',
            'Augmentation':'',
            'Evaluation':'',
        }
        data_records.append(row)

        # 요약 정보 출력
        print(f"Dataset: {dataset}")
        print(f"  사용 가능한 Split: {available_splits}")
        print(f"  사용 가능한 Split Num: {available_splits_nums}")
        print(f"  이미지 확장자: {str(dict(extension_counter_originals))}")
        print(f"  이미지 Shape: {original_shape_report}")
        print(f"  이미지 Range: {original_min_value} - {original_max_value}")
        print(f"  이미지 DataType: {original_dtype}")
        print(f"  마스크 Shape: {mask_shape_report}")
        print(f"  마스크 확장자: {str(dict(extension_counter_masks))}")
        print(f"  #Samples: {total_samples}")
        print(f"  Inchannels: {in_channels}")
        print(f"  #Classes: {total_classes}")
        print(f"  Unique Mask Classes: {sorted(list(all_classes))}")
        print("-" * 40)

    # DataFrame 생성 후 CSV로 저장
    df = pd.DataFrame(data_records)
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    print(f"\nSummary saved to CSV at: {output_csv_path}")
    return df

# Example usage
root_directory = './'  # Replace with your actual root directory path
output_csv = 'Final_Competition_Dataset_Inform.csv'  # Replace with the path where you want to save the CSV
dataset_list = ['VOC','ETIS','CVPPP','CFD','CarDD', ]  # 원하는 데이터셋 폴더명을 리스트로 입력
df = check_dataset_integrity_and_generate_csv(root_directory, output_csv, dataset_names=dataset_list)

Dataset: VOC - Unique mask classes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Available splits: ['train', 'val', 'test']
Dataset: VOC
  사용 가능한 Split: ['train', 'val', 'test']
  사용 가능한 Split Num: [1747, 583, 583]
  이미지 확장자: {'.png': 2913}
  이미지 Shape: {(256, 256, 3): 2913}
  이미지 Range: 0 - 255
  이미지 DataType: uint8
  마스크 Shape: {(256, 256, 1): 2913}
  마스크 확장자: {'.npy': 2913}
  #Samples: 2913
  Inchannels: 3
  #Classes: 21
  Unique Mask Classes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
----------------------------------------


Dataset: ETIS - Unique mask classes: [0, 1]
Available splits: ['train', 'val', 'test']
Dataset: ETIS
  사용 가능한 Split: ['train', 'val', 'test']
  사용 가능한 Split Num: [118, 39, 39]
  이미지 확장자: {'.png': 196}
  이미지 Shape: {(256, 256, 3): 196}
  이미지 Range: 0 - 255
  이미지 DataType: uint8
  마스크 Shape: {(256, 256, 1): 196}
  마스크 확장자: {'.npy': 196}
  #Samples: 196
  Inchannels: 3
  #Classes: 2
  Unique Mask Classes: [0, 1]
----------------------------------------


Dataset: CVPPP - Unique mask classes: [0, 1]
Available splits: ['train', 'val', 'test']
Dataset: CVPPP
  사용 가능한 Split: ['train', 'val', 'test']
  사용 가능한 Split Num: [486, 162, 162]
  이미지 확장자: {'.png': 810}
  이미지 Shape: {(256, 256, 3): 810}
  이미지 Range: 0 - 255
  이미지 DataType: uint8
  마스크 Shape: {(256, 256, 1): 810}
  마스크 확장자: {'.npy': 810}
  #Samples: 810
  Inchannels: 3
  #Classes: 2
  Unique Mask Classes: [0, 1]
----------------------------------------


Dataset: CFD - Unique mask classes: [0, 1]
Available splits: ['train', 'val', 'test']
Dataset: CFD
  사용 가능한 Split: ['train', 'val', 'test']
  사용 가능한 Split Num: [70, 24, 24]
  이미지 확장자: {'.png': 118}
  이미지 Shape: {(256, 256, 3): 118}
  이미지 Range: 0 - 247
  이미지 DataType: uint8
  마스크 Shape: {(256, 256, 1): 118}
  마스크 확장자: {'.npy': 118}
  #Samples: 118
  Inchannels: 3
  #Classes: 2
  Unique Mask Classes: [0, 1]
----------------------------------------


Dataset: CarDD - Unique mask classes: [0, 1]
Available splits: ['train', 'val', 'test']
Dataset: CarDD
  사용 가능한 Split: ['train', 'val', 'test']
  사용 가능한 Split Num: [310, 104, 104]
  이미지 확장자: {'.png': 518}
  이미지 Shape: {(256, 256, 3): 518}
  이미지 Range: 0 - 255
  이미지 DataType: uint8
  마스크 Shape: {(256, 256, 1): 518}
  마스크 확장자: {'.npy': 518}
  #Samples: 518
  Inchannels: 3
  #Classes: 2
  Unique Mask Classes: [0, 1]
----------------------------------------

Summary saved to CSV at: Final_Competition_Dataset_Inform.csv


In [17]:
# 시각화
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import shutil

# 클래스별 고유 색상 지정 (클래스 0은 배경, 나머지는 각 클래스에 고유 색상)
class_colors = {
    0: [0, 0, 0],         # 클래스 0: 검정색 (배경)
    1: [255, 0, 0],       # 클래스 1: 빨강
    2: [0, 255, 0],       # 클래스 2: 초록
    3: [0, 0, 255],       # 클래스 3: 파랑
    4: [255, 255, 0],     # 클래스 4: 노랑
    5: [255, 0, 255],     # 클래스 5: 분홍
    6: [0, 255, 255],     # 클래스 6: 하늘색
    7: [128, 0, 128],     # 클래스 7: 보라색
    8: [128, 128, 0],     # 클래스 8: 올리브색
    9: [0, 128, 128],     # 클래스 9: 청록색
    10: [128, 128, 128],  # 클래스 10: 회색
    11: [192, 0, 0],      # 클래스 11: 진한 빨강
    12: [0, 192, 0],      # 클래스 12: 진한 초록
    13: [0, 0, 192],      # 클래스 13: 진한 파랑
    14: [192, 192, 0],    # 클래스 14: 연한 노랑
    15: [192, 0, 192],    # 클래스 15: 연한 분홍
    16: [0, 192, 192],    # 클래스 16: 연한 하늘색
    17: [128, 64, 0],     # 클래스 17: 갈색
    18: [64, 128, 0],     # 클래스 18: 연두색
    19: [0, 64, 128],     # 클래스 19: 어두운 청록색
    20: [255, 128, 0],    # 클래스 20: 주황색
    21: [128, 0, 255],    # 클래스 21: 보라색 (밝은)
    22: [0, 128, 255],    # 클래스 22: 밝은 파랑
    23: [255, 128, 128],  # 클래스 23: 연한 빨강
    24: [128, 255, 128],  # 클래스 24: 연한 초록
    25: [128, 128, 255],  # 클래스 25: 연한 파랑
    26: [255, 255, 128],  # 클래스 26: 연한 노랑
    27: [255, 0, 128],    # 클래스 27: 핫핑크
    28: [128, 255, 0],    # 클래스 28: 형광 초록
    29: [0, 255, 128],    # 클래스 29: 밝은 청록색
}

# 클래스 마스크에 맞는 색상 적용 함수
def apply_class_colors(mask_img):
    # 컬러 마스크 이미지 생성
    height, width = mask_img.shape
    color_mask = np.zeros((height, width, 3), dtype=np.uint8)

    # 각 클래스에 해당하는 색상을 할당
    for class_value, color in class_colors.items():
        color_mask[mask_img == class_value] = color

    return color_mask

# 이미지와 마스크를 오버레이하여 시각화하는 함수 (클래스별 색상 적용)
def overlay_images_and_masks(original_img, mask_img, output_path):
    # 원본 이미지를 3채널로 변환 (필요한 경우)
    if original_img.ndim == 2:  # 1채널일 경우
        original_img = np.stack([original_img] * 3, axis=-1)
    
    # 마스크에 색상 적용
    mask_colored = apply_class_colors(mask_img)

    # 두 이미지를 오버레이 (50% 투명도)
    overlay = (0.6 * original_img / 255) + (0.4 * mask_colored / 255)

    # 결과 저장
    overlay_img = Image.fromarray((overlay * 255).astype(np.uint8))
    overlay_img.save(output_path)
    print(f"저장 완료: {output_path}")

    Image.fromarray(mask_colored.astype(np.uint8)).save(output_path.replace('_3.overlay','_2.mask'))

# 특정 split의 파일들을 처리하는 함수
def process_split(dataset_path, split_name, visualization_folder, max_files=50):
    """특정 split (train/val/test)의 파일들을 처리"""
    split_path = os.path.join(dataset_path, split_name)
    originals_folder = os.path.join(split_path, 'Originals')
    masks_folder = os.path.join(split_path, 'Masks')
    
    if not os.path.exists(originals_folder) or not os.path.exists(masks_folder):
        print(f"'{split_name}' split에 'Originals' 또는 'Masks' 폴더가 없어 스킵합니다.")
        return 0
    
    processed_count = 0
    split_viz_folder = os.path.join(visualization_folder, split_name)
    os.makedirs(split_viz_folder, exist_ok=True)
    
    # Originals 폴더 내 파일들을 처리 (최대 max_files개)
    original_files = [f for f in os.listdir(originals_folder) if f.endswith('.png')][:max_files]
    
    for file_name in original_files:
        original_path = os.path.join(originals_folder, file_name)
        mask_path = os.path.join(masks_folder, os.path.splitext(file_name)[0] + '.npy')

        if os.path.exists(mask_path):
            try:
                # 원본 이미지 및 마스크 불러오기
                original_img = np.array(Image.open(original_path))
                mask_img = np.load(mask_path)

                # 오버레이된 이미지 저장 경로
                output_path = os.path.join(split_viz_folder, f"{file_name}_3.overlay.png")

                # 오버레이 함수 호출
                shutil.copy(original_path, output_path.replace('_3.overlay','_1.original'))
                overlay_images_and_masks(original_img, mask_img, output_path)
                processed_count += 1
                
            except Exception as e:
                print(f"오버레이 실패: {split_name}/{file_name}, 에러: {e}")
        else:
            print(f"마스크 파일이 없습니다: {split_name}/{file_name}")
    
    return processed_count

# 데이터셋 폴더를 순회하며 오버레이를 생성하는 함수 (train/val/test 구조)
def process_datasets(root_folder, dataset_names=None, max_files_per_split=50, splits_to_process=['train', 'val', 'test']):
    """
    train/val/test 구조의 데이터셋들을 처리하는 함수
    
    Args:
        root_folder: 데이터셋들이 있는 루트 폴더
        dataset_names: 처리할 데이터셋 이름 리스트 (None이면 모든 데이터셋)
        max_files_per_split: 각 split에서 처리할 최대 파일 수
        splits_to_process: 처리할 split 리스트 (기본값: ['train', 'val', 'test'])
    """
    # 제공된 dataset_names가 없을 경우 모든 폴더를 대상으로 처리
    if dataset_names is None or len(dataset_names) == 0:
        datasets = [d for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]
    else:
        datasets = [d for d in dataset_names if os.path.isdir(os.path.join(root_folder, d))]

    # 각 데이터셋 순회
    for dataset in datasets:
        dataset_path = os.path.join(root_folder, dataset)
        print(f"\n=== 데이터셋 처리 시작: {dataset} ===")
        
        # 'Visualization' 폴더 생성
        visualization_folder = os.path.join(dataset_path, 'Visualization')
        os.makedirs(visualization_folder, exist_ok=True)
        
        total_processed = 0
        available_splits = []
        
        # 각 split (train/val/test)을 처리
        for split in splits_to_process:
            split_path = os.path.join(dataset_path, split)
            if os.path.exists(split_path):
                available_splits.append(split)
                print(f"  {split} split 처리 중...")
                processed_count = process_split(dataset_path, split, visualization_folder, max_files_per_split)
                total_processed += processed_count
                print(f"  {split} split 완료: {processed_count}개 파일 처리")
        
        if not available_splits:
            print(f"'{dataset}' 데이터셋에 유효한 split이 없어 스킵합니다.")
            continue
        
        print(f"=== 데이터셋 '{dataset}' 처리 완료 ===")
        print(f"  사용 가능한 splits: {available_splits}")
        print(f"  총 처리된 파일 수: {total_processed}개")

# 기존 구조 (Originals/Masks)도 지원하는 함수
def process_datasets_legacy(root_folder, dataset_names=None, max_files=50):
    """기존 구조 ({dataset}/Originals, {dataset}/Masks)를 처리하는 함수"""
    # 제공된 dataset_names가 없을 경우 모든 폴더를 대상으로 처리
    if dataset_names is None or len(dataset_names) == 0:
        datasets = [d for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]
    else:
        datasets = [d for d in dataset_names if os.path.isdir(os.path.join(root_folder, d))]

    # 각 데이터셋 순회
    for dataset in datasets:
        dataset_path = os.path.join(root_folder, dataset)
        
        # 각 데이터셋의 'Originals'와 'Masks' 폴더가 존재하는지 확인
        originals_folder = os.path.join(dataset_path, 'Originals')
        masks_folder = os.path.join(dataset_path, 'Masks')
        if not os.path.exists(originals_folder) or not os.path.exists(masks_folder):
            print(f"'{dataset}' 데이터셋에 'Originals' 또는 'Masks' 폴더가 없어 스킵합니다.")
            continue
        
        # 'Visualization' 폴더 생성
        visualization_folder = os.path.join(dataset_path, 'Visualization')
        os.makedirs(visualization_folder, exist_ok=True)
        
        # Originals 폴더 내 모든 파일을 확인
        for file_name in os.listdir(originals_folder)[:max_files]:
            if file_name.endswith('.png'):
                original_path = os.path.join(originals_folder, file_name)
                mask_path = os.path.join(masks_folder, os.path.splitext(file_name)[0] + '.npy')

                if os.path.exists(mask_path):
                    try:
                        # 원본 이미지 및 마스크 불러오기
                        original_img = np.array(Image.open(original_path))
                        mask_img = np.load(mask_path)

                        # 오버레이된 이미지 저장 경로
                        output_path = os.path.join(visualization_folder, f"{file_name}_3.overlay.png")

                        # 오버레이 함수 호출
                        shutil.copy(original_path, output_path.replace('_3.overlay','_1.original'))
                        overlay_images_and_masks(original_img, mask_img, output_path)
                        
                    except Exception as e:
                        print(f"오버레이 실패: {file_name}, 에러: {e}")
                else:
                    print(f"마스크 파일이 없습니다: {file_name}")

# 사용 예시
root_folder = './'  # Dataset들이 있는 상위 폴더 경로

# 처리할 데이터셋 리스트 (비어있으면 전체 데이터셋 처리)
dataset_names = ['VOC','ETIS','CVPPP','CFD','CarDD']  # 특정 데이터셋 이름 리스트 (비어있으면 전체 순회)

# train/val/test 구조로 처리 (새로운 방식)
print("=== Train/Val/Test 구조로 처리 시작 ===")
process_datasets(
    root_folder=root_folder, 
    dataset_names=dataset_names,
    max_files_per_split=2000,  # 각 split에서 최대 20개 파일 처리
    splits_to_process=['train', 'val', 'test']  # 처리할 split 지정
)


=== Train/Val/Test 구조로 처리 시작 ===

=== 데이터셋 처리 시작: VOC ===
  train split 처리 중...
저장 완료: ./VOC/Visualization/train/VOC_2011_002559.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2008_003874.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2010_001732.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2008_004212.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2007_002088.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2011_000573.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2009_004278.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2010_004419.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2009_000553.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2008_007945.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2011_002291.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2007_001487.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2009_004434.png_3.overlay.png
저장 완료: ./VOC/Visualization/train/VOC_2009_001332.